In [1]:
from darli.robots import Biped
from robot_descriptions import atlas_v4_description
import casadi as cs
import numpy as np


biped_urdf = atlas_v4_description.URDF_PATH

biped_model = Biped(
    biped_urdf,
    torso={"torso": "pelvis"},
    foots={"left_foot": "l_foot", "right_foot": "r_foot"},
)

In [3]:
nq = biped_model.nq  # dimensionality of configuration
nv = biped_model.nv  # dimensionality of generilized velocities
nu = biped_model.nu  # dimensionality  of control inputs
q_min, q_max = (
    biped_model.q_min,
    biped_model.q_max,
)  # minimal and maximal limits on joint pos
nq, nv, nu

(30, 30, 24)

In [4]:
jacobians = []
jacobians_dt = []
for body in {"right_foot", "left_foot"}:
    contact_dim = biped_model.body(body).contact.dim
    jacobians.append(biped_model.body(body).contact.jacobian(biped_model._model._q))
    jacobians_dt.append(
        biped_model.body("right_foot")
        .jacobian_dt()
        .world(biped_model._model._q, biped_model._model._v)[:contact_dim, :]
    )


jacobian_matrix = cs.vertcat(*jacobians)
jacobian_dt = cs.vertcat(*jacobians_dt)

In [5]:
M = biped_model.inertia(biped_model._model._q)
h = biped_model.bias_force(biped_model._model._q, biped_model._model._v)

In [6]:
nj = jacobian_matrix.rows()

In [7]:
#
kkt_matrix = cs.blockcat(M, -jacobian_matrix.T, jacobian_matrix, np.zeros((nj, nj)))
kkt_rhs = cs.vertcat(biped_model._qfrc_u - h, -jacobian_dt @ biped_model._model._v)

In [8]:
kkt_solution = cs.solve(kkt_matrix, kkt_rhs)

In [9]:
ddq_minimal = kkt_solution[: biped_model.nv]
contact_forces = kkt_solution[biped_model.nv :]

In [10]:
fw_dyn = cs.Function(
    "reduced_forward_dynamics",
    [biped_model._model._q, biped_model._model._v, biped_model._u],
    [ddq_minimal],
    ["q", "v", "u"],
    ["ddq_reduced"],
)

In [11]:
con_force = cs.Function(
    "contact_forces",
    [biped_model._model._q, biped_model._model._v, biped_model._u],
    [contact_forces],
    ["q", "v", "u"],
    ["constraint_forces"],
)

In [12]:
con_force

Function(contact_forces:(q[30],v[30],u[24])->(constraint_forces[12]) SXFunction)

In [13]:
# biped_model.state_space._state

In [14]:
# fw_dyn(biped_model._model._q, biped_model._model._v, biped_model._u)*cs.SX.sym('dt', 1)

In [15]:
def func(q, v, u, dt):
    dv = fw_dyn(q, v, u)
    v_next = v + dv * dt
    # print(v_next)
    q_next = biped_model._model._kindyn.integrate()(q, v_next * dt)
    return cs.vertcat(q_next, v_next)


dt = cs.SX.sym("dt", 1)

discrete_state_space = cs.Function(
    "discrete_state_space",
    [biped_model.state_space._state, biped_model._u, dt],
    [func(biped_model._model._q, biped_model._model._v, biped_model._u, dt)],
    ["state", "control", "dt"],
    ["next_state"],
)

In [16]:
discrete_state_space

Function(discrete_state_space:(state[60],control[24],dt)->(next_state[60]) SXFunction)

In [17]:
%timeit discrete_state_space()

118 µs ± 1.27 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [18]:
%timeit fw_dyn()

115 µs ± 769 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
